In [1]:
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
X_train = pd.read_csv('X_train1.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train1.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test1.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test1.csv')
y_test=y_test.to_numpy().reshape(-1)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
import os
from threading import Thread

In [5]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [6]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 7

In [7]:
time.sleep(15)

In [8]:
results={}
def train_log_reg(X_train,y_train):

  train_time_start=time.time()
  model = LogisticRegression(max_iter=3, solver='saga',C=100)
  model.fit(X_train, y_train)
  train_time_end=time.time()

  results['model']=model
  results['train_time']=train_time_end-train_time_start
  results['weights']=model.coef_
  results['bias']=model.intercept_

In [9]:
train_thread = Thread(target=train_log_reg, args=(X_train, y_train))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 1.0% | RAM utilization: 1.5513985219473216%
CPU utilization: 0.0% | RAM utilization: 1.5513985219473216%
Total CPU utilization over 7 seconds: 6.0%
CPU usage per second: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
RAM usage per second: [1.5513985219473216, 1.5513985219473216, 1.5513985219473216, 1.5513985219473216, 1.5513985219473216, 1.5513985219473216, 1.5513985219473216]
Total CPU resource consumption: 0.06 CPU-seconds


In [10]:
print(results['weights'])

[[ 0.09043485  0.13117685 -0.1956727   0.14859336 -0.40530515 -0.00646334
   0.00403697 -0.19450672  0.7968146   0.12947169  0.33331971  0.4159908
   0.36464387  0.09512577  0.13774261  0.20521496  0.00437713  0.02397745
  -0.13112346 -0.61244978 -0.75320168 -0.30729143]]


In [11]:
print(results['bias'])

[0.24479839]


In [12]:
results['train_time']

0.013590335845947266

In [13]:
test_time_start=time.time()
y_pred=results['model'].predict(X_test)
test_time_end=time.time()

In [14]:
results['test_time']=test_time_end-test_time_start
results['test_time']

0.0008063316345214844

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8


In [16]:
import joblib
joblib.dump((results['weights'][0], results['bias'][0], accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_sklearn.pkl')

['variables_sklearn.pkl']